# Stochastic contribution analysis for a parameterized foreground system

Pierre Jouannais

If you are studying the uncertain contribution in a foreground system for which you have a model that can simulate the inventory based on different parameters (process simulation etc.).

In [1]:
import bw2data
import bw2io
from bw2data.parameters import *
import brightway2 as bw
import time
import numpy as np
import pandas as pd


I have used Salib to create my sample of parameters, but it can be done without.
Sensitivity analysis can be done at the same time very easily.

In [137]:

from SALib.sample import saltelli
from SALib.sample import fast_sampler
from SALib.analyze import sobol
from SALib.analyze import fast
import SALib

## Loading databases

In [9]:
bw.projects.set_current('Eco38_4') 

In [98]:

# Loading Ecoinvent
Ecoinvent = bw.Database('ecoinvent 3.8 conseq')
biosph = bw.Database('biosphere3')


Here I am making a dummy foreground product system for the example.

In [138]:
act1=Ecoinvent.random()
act2=Ecoinvent.random()
act3=Ecoinvent.random()

We don't care about the amounts in the initial foreground as we will modify them according to our own model.

In [93]:
db_contrib_Magic = bw.Database('db_contrib_Magic')
db_contrib_Magic.write({  
    ('db_contrib_Magic', 'magic_prod_for_sa'): {
        'name': 'Magic Prod',
        'unit': 'Magic Unit',
        'exchanges': [{                  
            'input': ('db_contrib_Magic', 'First activity'),   
            'amount': 0.2, 
            'type': 'technosphere'},  
        { 
            'input': ('db_contrib_Magic', 'Second activity'), 
            'amount': 0.2,  
            'type': 'technosphere'},    
 
        { 
            'input':  ('db_contrib_Magic', 'Third activity'), 
            'amount': -0.0000002, 
            'type': 'technosphere'},  

        {
            'input': ('db_contrib_Magic', 'magic_prod_for_sa'),
            'amount': 1.0, 
            'type': 'production'}]
    },
    ('db_contrib_Magic', 'First activity'): {
        'name': 'First activity',
        'unit': 'Strange Unit',
        'exchanges': [{
            'input': act1,  
            'amount': 1,
            'type': 'technosphere'},  
            {'input': ('db_contrib_Magic', 'First activity'),
            'amount': 1.0, 
            'type': 'production'}]},
    
    ('db_contrib_Magic', 'Second activity'): {
        'name': 'Second activity',
        'unit': 'Strange Unit',
        'exchanges': [{
            'input': act2,  
            'amount': 1, 
            'type': 'technosphere'},  
   
        {
            'input': ('db_contrib_Magic', 'Second activity'),
            'amount': 1.0,
            'type': 'production'}]},
    
    ('db_contrib_Magic', 'Third activity'): {
        'name': 'Third activity',
        'unit': 'Strange Unit',
        'exchanges': [{
            'input': act3, 
            'amount': 1, 
            'type': 'technosphere'},  
   
        {
            'input': ('db_contrib_Magic','Third activity'),
            'amount': 1.0, 
            'type': 'production'}]}})

Writing activities to SQLite3 database:
0% [####] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 02/07/2023 16:56:49
  Finished: 02/07/2023 16:56:49
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 0.66


In [139]:
for act in db_contrib_Magic:
    print(act)

'Magic Prod' (Magic Unit, None, None)
'Third activity' (Strange Unit, None, None)
'First activity' (Strange Unit, None, None)
'Second activity' (Strange Unit, None, None)


 We collect the main activity for which we want to assess the contribution of the different inputs.

In [140]:
for act in db_contrib_Magic:
    
    if act['name'] == 'Magic Prod':
        
        Mainprod = db_contrib_Magic.get(act['code'])   


We collect the input activities to our main process and save it as a list of functional units for LCA.

***Important : You first need to "Flatten" your product system. We are assessing the contribution of the activites in THE FIRST LAYER of the product system = the direct inputs to the main activity. There are no emissions associated to the main activity and there are no exchanges between the activities of the first layer.***

In [143]:
list_foreground_technosphere_inputs_FU = []

list_foreground_technosphere_inputs_names = []

for exc in list(Mainprod.exchanges()):

        if exc['type']!='production': # We just want the inputs to the main process
            
            exchange1 = db_contrib_Magic.get(exc['input'][1])   
            
            name_exchange = exchange1['name']  
            
            list_foreground_technosphere_inputs_names.append(name_exchange)
        
            list_foreground_technosphere_inputs_FU.append({exchange1 : 1})
    

In [145]:
# Choose methods

methods_selected = ([
     ('ReCiPe Midpoint (H) V1.13', 'terrestrial ecotoxicity', 'TETPinf'),
     ('ReCiPe Midpoint (H) V1.13', 'climate change', 'GWP100'),
     ('ReCiPe Midpoint (H) V1.13', 'freshwater eutrophication', 'FEP'),
     ('ReCiPe Midpoint (H) V1.13', 'water depletion', 'WDP')])



Calculate impact for each 1 unit of each input and for each method and save it in a dictionnary

In [147]:
my_calculation_setup = {'inv': list_foreground_technosphere_inputs_FU, 'ia': methods_selected}

bw.calculation_setups['technosphere_inputs'] = my_calculation_setup



# Calculating the impacts for all chosen methods

mlca_techno = bw.MultiLCA('technosphere_inputs')  

res = mlca_techno.results

dict_mono_technosphere_lcas = { name : results for (name, results) in zip(list_foreground_technosphere_inputs_names,res) }






In [148]:
dict_mono_technosphere_lcas

{'First activity': array([0., 0., 0., 0.]),
 'Second activity': array([8.71268255e-06, 6.42452351e-02, 2.49108858e-05, 2.11567743e-04]),
 'Third activity': array([6.82400124e-01, 5.07709581e+03, 1.16517033e+00, 5.79338685e+00])}

In [149]:

# Each flow has the same name as the corresponding activity in the original database

names_act = ['First activity','Second activity','Third activity']


# Assigning each process to a a broader category for contribution analysis if needed

# Names of categories

categories_contribution = ['Thermoregulation','Cleaning']


# List of processes to assign to categories (same order)

processes_in_categories = [['First activity', 'Second activity'],  # First and second acitivites belong to "Thermoregulation"
                           ['Third activity']] # Third Activity is alone in "Cleaning"






There are 2 uncertain parameters in our model and we assume normal distributions for them

In [152]:
names_param = ["param1","param2"] 
bounds = [[5,0.2],[3,0.8]]
dists=["norm","norm"]
size=30

We generate the stochastic sample with Salib but it's not necessary

In [156]:
problem = {'num_vars': len(names_param),  # number of variables
           'names': names_param,  
           'bounds': bounds,
           'dists': dists}

30 Montecarlo iterations

In [155]:
sample = SALib.sample.saltelli.sample(problem,
                                          size,
                                          calc_second_order=False)

In [109]:
sample

array([[4.84537659, 1.95943582],
       [5.0093053 , 1.95943582],
       [4.84537659, 3.36692128],
       [5.0093053 , 3.36692128],
       [5.11640588, 3.1958097 ],
       [4.58308826, 3.1958097 ],
       [5.11640588, 2.25776663],
       [4.58308826, 2.25776663],
       [5.37535801, 2.6845598 ],
       [5.14681907, 2.6845598 ],
       [5.37535801, 4.16164958],
       [5.14681907, 4.16164958],
       [4.98480855, 3.81783714],
       [4.87656199, 3.81783714],
       [4.98480855, 2.8522923 ],
       [4.87656199, 2.8522923 ],
       [4.92008053, 2.94316142],
       [4.78710315, 2.94316142],
       [4.92008053, 2.58451878],
       [4.78710315, 2.58451878],
       [5.20281504, 4.52485908],
       [5.07359534, 4.52485908],
       [5.20281504, 3.67833334],
       [5.07359534, 3.67833334],
       [5.04794412, 2.3867738 ],
       [4.94598677, 2.3867738 ],
       [5.04794412, 1.69756506],
       [4.94598677, 1.69756506],
       [4.73756027, 3.47027089],
       [5.24913125, 3.47027089],
       [4.

The foreground inventory (amounts of first, second and third activities) is calculated based on a model of some type (Process simulation, aspen, anything)

In [157]:
def dummy_model_calculate_inputs(param1,param2):
    
    input1 = param1*2000/(1+param2)  # Just dummy relationships between parameters and amounts of inputs
    input2 = param2/param1*2
    input3 = input2/input1
    
    return([input1,input2,input3])

## Calculate everything

Initialize the lists and tables that will collect the results.

In [124]:
results_table = np.empty((0,
                          len(names_param)+len(names_act) + len(methods_selected)),
                         dtype=float)


# list of tables whih will contain the conribution of each process category to each impact category

# There are the same. I will just calculate contributions in two different ways
list_tables_contribution = [np.zeros((len(sample),
                                      len(categories_contribution)),
                                     dtype=float) for i in range(len(methods_selected))]  

list_tables_contribution_abs = [np.zeros((len(sample),
                                      len(categories_contribution)),
                                     dtype=float) for i in range(len(methods_selected))]  

In [125]:
count = -1 # an index to put the results in the right position

for rowparam in sample: # We browse the stochastic sample of parameters.
    
    # rowparam is a set of parameters
    count += 1
    
    LCI_collected = calculate_inputs(rowparam[0],rowparam[1]) # Collect output of the model which calculates the LCI
    
    LCIdict_collected = {a: i for a,i in zip(names_act,LCI_collected)} # Put it in a dictionnary with the names of the activities as keys

    dict_mono_technosphere_impacts = dict_mono_technosphere_lcas.copy() # I just want to keep the structure so I make a copy
    
    for i in dict_mono_technosphere_lcas: # Impact associated with an input = amount of this input * impact for 1 unit of this input 
            
        dict_mono_technosphere_impacts[i]=[LCIdict_collected[i]*a for a in dict_mono_technosphere_lcas[i]] # Done for all the methods at the same time
    
    
    # Calculating total impact by summing
        
    list_LCA_res =[]  # Will contain total impact for the main activity, for all the methods 

    for meth_index in range(len(methods_selected)):

        sum_impact = sum([dict_mono_technosphere_impacts[flow][meth_index] for flow in dict_mono_technosphere_impacts ])


        list_LCA_res.append(sum_impact)
  

    # The row I will add to the final result
    row_to_add = list(rowparam) + LCI_collected  + list_LCA_res 

    # Adding this new row

    results_table = np.vstack((results_table, row_to_add))  

    names_methods_adjusted = [a[-1] for a in methods_selected]

    names_for_df = names_param+names_act + names_methods_adjusted

    results_table_df = pd.DataFrame(results_table, columns=names_for_df)


    # Contribution per process category
    for process in dict_mono_technosphere_impacts :

        # browsing the categories
        for index_content_categ in range(len(processes_in_categories)):


            # if this process belongs to category
            if process in processes_in_categories[index_content_categ]:

                # Then we add this value to the corresponding colum in the  list_tables_contribution
                for meth_index in range(len(methods_selected)): #we do this for all methods 

                    list_tables_contribution[meth_index][count, index_content_categ] = (
                        list_tables_contribution[meth_index][count, index_content_categ] 
                        + dict_mono_technosphere_impacts[process][meth_index])


results_table_df contains the parameters values, the corresponding amounts of inputs, and the total impact for the activity

In [126]:
results_table_df

,param1,param2,First activity,Second activity,Third activity,TETPinf,GWP100,FEP,WDP
0,4.845377,1.959436,3274.527229,0.808786,0.000247,0.000176,1.305968,0.000308,0.001602
1,5.009305,1.959436,3385.310981,0.782318,0.000231,0.000165,1.223537,0.000289,0.001504
2,4.845377,3.366921,2219.127061,1.389746,0.000626,0.000439,3.268856,0.000764,0.003922
3,5.009305,3.366921,2294.204532,1.344267,0.000586,0.000412,3.061237,0.000716,0.003679
4,5.116406,3.195810,2438.816939,1.249240,0.000512,0.000360,2.680909,0.000628,0.003232
...,...,...,...,...,...,...,...,...,...
115,5.185182,3.227245,2453.220529,1.244795,0.000507,0.000357,2.656155,0.000622,0.003203
116,5.231499,3.002937,2613.829961,1.148022,0.000439,0.000310,2.303669,0.000540,0.002787
117,4.907998,3.002937,2452.198019,1.223691,0.000499,0.000351,2.612180,0.000612,0.003150
118,5.231499,2.026460,3457.173542,0.774715,0.000224,0.000160,1.187494,0.000280,0.001462


Now we calculate the contributions based 

In [159]:

#Calulating contribution sum
for index_method in range(len(methods_selected)):

    for index_row in range(len(sample)):

        sumrow = np.sum(list_tables_contribution[index_method][index_row])

        sumrow_abs = sum([abs(a) for a in list_tables_contribution[index_method][index_row,:]]) # Here I calculate the contribution based on the absolute values because I think it makes more sense when you can have postitive and negative numbers 

        for index_col in range(len(categories_contribution)):

            list_tables_contribution_abs[index_method][index_row][index_col] =(
                list_tables_contribution[index_method][index_row][index_col]
                *100/sumrow_abs)

            list_tables_contribution[index_method][index_row][index_col] =(
                list_tables_contribution[index_method][index_row][index_col]
                *100/sumrow)



#  Conversion to Dataframes
list_tables_contribution_df = [pd.DataFrame(
    table, columns=categories_contribution) for table in list_tables_contribution]

list_tables_contribution_abs_df=[pd.DataFrame(
    table, columns=categories_contribution) for table in list_tables_contribution_abs]



In [2]:
list_tables_contribution_abs_df # A list of tables of contributions for all MC iterations. 1 table per impact category.

NameError: name 'list_tables_contribution_abs_df' is not defined

In [131]:
list_tables_contribution_abs_df[0]

,Thermoregulation,Cleaning
0,4.013041,95.986959
1,4.143185,95.856815
2,2.755251,97.244749
3,2.845814,97.154186
4,3.019779,96.980221
...,...,...
115,3.037072,96.962928
116,3.229484,96.770516
117,3.035845,96.964155
118,4.227418,95.772582


In [1]:
list_tables_contribution_abs_df[3].plot.kde()  # The fourth impact category. Feel free to plot something better than this.

NameError: name 'list_tables_contribution_abs_df' is not defined